In [ ]:
!pip install scikit-image matplotlib

In [ ]:
!pip install xgboost

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import io # Input/Output Module
import os # OS interfaces
import cv2 # OpenCV package
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import floor, ceil
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.manifold import TSNE
import seaborn as sns

from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library


Computer Vision
---------------------------------------------------------------

The goal of this notebook is to explore advanced techniques for constructing features that better describe objects of interest and to perform face recognition using these features.

---------------------------------------------------------------
This notebook is structured as follows:
0. Data loading & Preprocessing
1. Feature Representations
2. Evaluation Metrics 
3. Classifiers
4. Experiments
5. Publishing best results
6. Discussion

---------------------------------------------------------------
# 0. Data loading & Preprocessing

## 0.1. Loading data
The training set is many times smaller than the test set and this might strike you as odd, however, this is close to a real world scenario where the system might be put through daily use! In this notebook, I will try to do the best I can with the data that I've got! 

use a blacklist to remove the bad images from te training set.

In [ ]:
# Input data files are available in the read-only "../input/" directory

train = pd.read_csv(
    '/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/train_set.csv', index_col = 0)
train.index = train.index.rename('id')

test = pd.read_csv(
    '/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/test_set.csv', index_col = 0)
test.index = test.index.rename('id')

# read the images as numpy arrays and store in "img" column
train['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in train.iterrows()]

test['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in test.iterrows()]
  

train_size, test_size = len(train),len(test)

"The training set contains {} examples, the test set contains {} examples.".format(train_size, test_size)

*Note: this dataset is a subset of the* [*VGG face dataset*](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/).

## 0.2. A first look
Let's have a look at the data columns and class distribution.

In [ ]:
# The training set contains an identifier, name, image information and class label
train.head(1)

In [ ]:
# The test set only contains an identifier and corresponding image information.
test.head(1)

In [ ]:
# The class distribution in the training set:
train.groupby('name').agg({'img':'count', 'class': 'max'})

Note that **Jesse is assigned the classification label 1**, and **Mila is assigned the classification label 2**. The dataset also contains 20 images of **look alikes (assigned classification label 0)** and the raw images. 

## 0.3. Preprocess data
### 0.3.1 Example: HAAR face detector
In the initial template of the notebook [HAAR feature based cascade classifiers](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html) was used to detect faces. The faces were resized so that they all have the same shape. If there are multiple faces in an image, it would only take the first one.

In [ ]:
class HAARPreprocessor():
    """Preprocessing pipeline built around HAAR feature based cascade classifiers. """
    
    def __init__(self, path, face_size):
        self.face_size = face_size
        file_path = os.path.join(path, "haarcascade_frontalface_default.xml")
        if not os.path.exists(file_path): 
            if not os.path.exists(path):
                os.mkdir(path)
            self.download_model(file_path)
        
        self.classifier = cv2.CascadeClassifier(file_path)
  
    def download_model(self, path):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/"\
            "haarcascades/haarcascade_frontalface_default.xml"
        
        with request.urlopen(url) as r, open(path, 'wb') as f:
            f.write(r.read())
            
    def detect_faces(self, img):
        """Detect all faces in an image."""
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.classifier.detectMultiScale(
            img_gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
    def extract_faces(self, img):
        """Returns all faces (cropped) in an image."""
        
        faces = self.detect_faces(img)

        return [img[y:y+h, x:x+w] for (x, y, w, h) in faces]
    
    def preprocess(self, data_row):
        faces = self.extract_faces(data_row['img'])
        
        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.empty(self.face_size + (3,))
            nan_img[:] = np.nan
            return nan_img
        
        # only return the first face
        return cv2.resize(faces[0], self.face_size, interpolation = cv2.INTER_AREA)
            
    def __call__(self, data):
        return np.stack([self.preprocess(row) for _, row in data.iterrows()]).astype(int)

### 0.3.1 Improvement: HAAR face detector

While the code from HAARPreprocessor worked, it did not result into the wanted result for every trainings example. That's why the code is adjusted in the class HAARPreprocessorMultiple, which had better results in combination with futher proccesing.

In [ ]:
class HAARPreprocessorMultiple():
    """Preprocessing pipeline built around HAAR feature based cascade classifiers. """
    
    def __init__(self, path, face_size):
        self.face_size = face_size
        file_path = os.path.join(path, "haarcascade_frontalface_default.xml")
        if not os.path.exists(file_path): 
            if not os.path.exists(path):
                os.mkdir(path)
            self.download_model(file_path)
        
        self.classifier = cv2.CascadeClassifier(file_path)
  
    def download_model(self, path):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/"\
            "haarcascades/haarcascade_frontalface_default.xml"
        
        with request.urlopen(url) as r, open(path, 'wb') as f:
            f.write(r.read())
            
    def detect_faces(self, img):
        """Detect all faces in an image."""
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.classifier.detectMultiScale(
            img_gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
    def extract_faces(self, img):
        """Returns all faces (cropped) in an image."""
        
        faces = self.detect_faces(img)

        return [img[y:y+h, x:x+w] for (x, y, w, h) in faces]
    
    # preprocesses and returns first five faces
    def preprocess(self, data_row):
        faces = self.extract_faces(data_row['img'])
        
        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.empty(self.face_size + (3,))
            nan_img[:] = np.nan
            return [nan_img]
        
        return [cv2.resize(face, self.face_size, interpolation = cv2.INTER_AREA) for face in faces]
            
    def __call__(self, data):
        return [self.preprocess(row) for _, row in data.iterrows()]

### 0.3.1 (No Improvement): DNN Face Detector  

Initially, the results of the HAAR detector for face extraction were deemed unsatisfactory, prompting an exploration of the DNN face detector as an alternative. However, this approach did not yield an improved score. Consequently, HAARPreprocessorMultiple was selected as the final preprocessor.


In [ ]:
# net = cv2.dnn.readNetFromCaffe("/kaggle/input/mod-face/deploy.prototxt", "/kaggle/input/mod-face/res10_300x300_ssd_iter_140000.caffemodel")

In [ ]:
class DNNPreprocessor():
    """Preprocessing pipeline built around DNN based face detection. """

    def __init__(self, net, face_size):
        self.face_size = face_size
        self.net = net

    def detect_faces(self, img):
        """Detect all faces in an image."""
        # Get image dimensions
        (h, w) = img.shape[:2]
        # Preprocess the image
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        # Pass the blob through the network and perform inference
        self.net.setInput(blob)
        detections = self.net.forward()
        # Initialize a list to store bounding box coordinates
        bounding_boxes = []
        # Loop over the detections
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            # Filter out weak detections
            if confidence > 0.9:
                # Compute the bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                # Add the bounding box coordinates to the list
                bounding_boxes.append((startX, startY, endX, endY))
        return bounding_boxes

    def cut_out_faces(self, img, bounding_boxes):
        """Returns all faces (cropped) in an image."""
        # Initialize a list to store cropped faces
        cropped_faces = []
        # Loop through each bounding box
        for (startX, startY, endX, endY) in bounding_boxes:
            # Calculate the width and height of the bounding box
            width = endX - startX
            height = endY - startY

            # Calculate the size of the margin (10% of the width/height)
            margin = int(0.1 *  height)

            # Calculate new start and end coordinates for the cropped face
            new_startX = max(0, startX - margin)
            new_startY = max(0, startY - margin)
            new_endX = min(img.shape[1], endX + margin)
            new_endY = min(img.shape[0], endY + margin)

            # Ensure the cropped region is square
            square_size = min(new_endY - new_startY, new_endX - new_startX)
            new_endX = new_startX + square_size
            new_endY = new_startY + square_size

            # Crop the face from the image
            face = img[new_startY:new_endY, new_startX:new_endX]
            face = cv2.resize(face, (100, 100))
            # Append the cropped face to the list
            cropped_faces.append(face)

        return cropped_faces

    def preprocess(self, data_row):
        faces = self.cut_out_faces(data_row['img'], self.detect_faces(data_row['img']))

        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.full(self.face_size + (3,), -1)  # Fill with -1 instead of np.nan
            return nan_img

        # only return the first face
        return cv2.resize(faces[0], self.face_size, interpolation = cv2.INTER_AREA)

    def __call__(self, data):
        return np.stack([self.preprocess(row) for _, row in data.iterrows()]).astype(int)

**Preprocces data**

In [ ]:
# parameter to play with 
FACE_SIZE = (100, 100)

def plot_image_sequence(data, n, imgs_per_row=7):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i])
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i])
        else:
            ax[int(i%n)].imshow(data[i])
    plt.show()

    
# preprocessed data 
# preprocessor = DNNPreprocessor(net, face_size=FACE_SIZE)
preprocessor = HAARPreprocessorMultiple(path = '../../tmp', face_size=FACE_SIZE)

train_X_faces, train_y = preprocessor(train), train['class'].values

**Further proccesing**

Analyzing the initial results of the preprocessor revealed instances where the wrong face was extracted from the training images or no face was detected at all. The following code blocks address these issues by manually removing incorrect extractions or selecting the correct face.


In [ ]:
preprocessorMultiple = HAARPreprocessorMultiple(path = '../../tmp', face_size=FACE_SIZE)

test_X_faces = preprocessorMultiple(test)

In [ ]:
# 14, 35, 39, 40
train_X_faces[14] = [cv2.resize(train["img"][14][80:240,70:230,:], (100, 100), interpolation = cv2.INTER_LINEAR)]
train_X_faces[35] = [cv2.resize(train["img"][35][50:130,190:260,:], (100, 100), interpolation = cv2.INTER_LINEAR)]
train_X_faces[40] = [cv2.resize(train["img"][40][80:260,100:220,:], (100, 100), interpolation = cv2.INTER_LINEAR)]

In [ ]:
train_X = []
for i, row in enumerate(train_X_faces):
    if i == 23:
        train_X.append(row[1])
    elif i == 24:
        train_X.append(row[1])
    elif i == 28:
        train_X.append(row[2])
    elif i == 49:
        train_X.append(row[1])
    elif i == 70:
        train_X.append(row[2])
    else:
        train_X.append(row[0])

In [ ]:
# remove images from the trainings data that don't contain information
blacklist = [65]

train_X = np.delete(train_X, blacklist, axis=0)
train_y = np.delete(train_y, blacklist, axis=0)

In [ ]:
train_X = np.array(train_X).astype(int)

**Visualise**

Now that the data is cleaned, lets verify it by visually looking at the trainings examples for each class.
Let's plot a few examples.

In [ ]:
# plot faces of Michael and Sarah
faces = train_X[train_y == 0]
plot_image_sequence(faces, n=len(faces), imgs_per_row=10)

In [ ]:
# plot faces of Jesse
faces = train_X[train_y == 1]
plot_image_sequence(faces, n=len(faces), imgs_per_row=10)

In [ ]:
# plot faces of Mila
faces = train_X[train_y == 2]
plot_image_sequence(faces, n=len(faces), imgs_per_row=10)

**Data augmentation**

Random data augmentation is added to make the data more optimal for the models.
Based on:
* gaussian noise
* random erasing
* horizontal flip
* adjusting the brightness

It is determined randomly when these concepts are applied.

In [ ]:
def add_gaussian_noise(image,p=0.5, mean=0, std=25):
    if random.uniform(0, 1) > p:
        return image
    """Add Gaussian noise to an image."""
    row, col, ch = image.shape
    gauss = np.random.normal(mean, std, (row, col, ch))
    noisy_image = np.clip(image + gauss, 0, 255)
    noisy_image = noisy_image.astype(np.uint8)
    return noisy_image

In [ ]:
def random_erasing(image, p=0.75, sl=0.02, sh=0.4, r1=0.3, r2=3):
    if random.uniform(0, 1) > p:
        return image

    img_h, img_w, _ = image.shape
    area = img_h * img_w

    while True:
        target_area = random.uniform(sl, sh) * area
        aspect_ratio = random.uniform(r1, r2)
        w = int(np.sqrt(target_area * aspect_ratio))
        h = int(np.sqrt(target_area / aspect_ratio))
        left = random.randint(0, img_w)
        top = random.randint(0, img_h)

        if left + w <= img_w and top + h <= img_h:
            erased_image = np.copy(image)
            erased_image[top:top + h, left:left + w, :] = np.random.rand(h, w, 3) * 255
            return erased_image

In [ ]:
def horizontal_flip(image,p=0.25):
    if random.uniform(0, 1) > p:
        return image
    return np.fliplr(image)

In [ ]:
def adjust_brightness(image, probability=1, brightness_range=(0.7, 1.3), contrast_range=(0.7, 1.3)):
    if random.uniform(0, 1) > probability:
        return image
    brightness_factor = random.uniform(*brightness_range)
    contrast_factor = random.uniform(*contrast_range)
    adjusted_image = cv2.convertScaleAbs(image, alpha=contrast_factor, beta=brightness_factor)

    return adjusted_image

In [ ]:
def random_augmentations(image,p=0.25):
    image = add_gaussian_noise(image)
    image = random_erasing(image)
    image = horizontal_flip(image)
    image = adjust_brightness(image)
    return image

## 0.4. Store Preprocessed data (optional)
<div class="alert alert-block alert-info">
<b>NOTE:</b> You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All". Feel free to use this to store intermediary results.
</div>

In [ ]:
# save preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
    
# np.save(os.path.join(prep_path, 'train_X.npy'), train_X)
# np.save(os.path.join(prep_path, 'train_y.npy'), train_y)
# np.save(os.path.join(prep_path, 'test_X.npy'), test_X)

# load preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
# train_X = np.load(os.path.join(prep_path, 'train_X.npy'))
# train_y = np.load(os.path.join(prep_path, 'train_y.npy'))
# test_X = np.load(os.path.join(prep_path, 'test_X.npy'))

# 1. Feature Representations
## 1.0. Example: Identify feature extractor
The example feature extractor doesn't actually do anything... It just returns the input:
$$
\forall x : f(x) = x.
$$

It does make for a good placeholder and baseclass.

In [ ]:
class IdentityFeatureExtractor:
    """A simple function that returns the input"""
    
    def transform(self, X):
        return X
    
    def __call__(self, X):
        return self.transform(X)

## 1.1. Baseline 1: HOG feature extractor/Scale Invariant Feature Transform

### HOG Feature Extractor

The Histogram of Oriented Gradients (HOG) is a feature descriptor that analyzes the distribution of gradient orientations within localized regions of an image. This method primarily captures the structural and shape-related characteristics of objects.

The process begins by dividing the image into small, non-overlapping cells. The selection of cell size is a key design choice; in this case, 8x8 pixel cells were used, with the image resized to 64x64 pixels.

Next, the image gradient is computed, derived from both the magnitude and angle of intensity changes at each pixel. These gradients are then grouped into 8x8 cells to form blocks. Within each block, a 9-bin histogram is generated, where each bin represents a 20-degree range of gradient angles.

The cells are further organized into larger blocks of equal size, with 2x2 blocks utilized in this implementation.

Finally, block-wise normalization is applied to minimize the impact of contrast variations across images of the same object, improving the robustness of the extracted features.


In [ ]:
from skimage.feature import hog

class HOGFeatureExtractor(IdentityFeatureExtractor):
    
    def __init__(self, **params):
        self.params = params
        self.histograms = []
        self.visual_representation = []
        self.tsne_transformed_data = None
        
        
    def transform(self, X):        
        for instance in X:
            instance = np.array(instance, dtype='uint8')
            instance_rsz = cv2.resize(instance, (64, 64))
            instance_rsz_gray = cv2.cvtColor(instance_rsz, cv2.COLOR_BGR2GRAY)
            
            hist, hog_image = hog(instance_rsz_gray, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
            
            self.histograms.append(hist)
            self.visual_representation.append(hog_image)

        return self.histograms
    
    def tsne_transform(self, n_components=2):
        tsne = TSNE(n_components=n_components, learning_rate='auto', init='pca', random_state = 26)
        self.tsne_transformed_data = tsne.fit_transform(np.array(self.histograms))
        


The hog function from skimage is used to compute the features. This function comes with a added result which can be used to visualize the results.

The following transformations are applied on the data:
* Resize the image to 64x64
* Convert the image to grayscale
* Apply the HOG algorithm with the parameters specified above, the output of this functions are the features computed by HOG and a visualization of it.


In [ ]:
hog_feature_extractor = HOGFeatureExtractor()
hists = hog_feature_extractor.transform(train_X)

In [ ]:
plot_image_sequence(hog_feature_extractor.visual_representation, n=50, imgs_per_row=10)

### 1.1.1. t-SNE Plots
t-SNE stands for t-Distributed Stochastic Neighbor Embedding. It is a machine learning algorithm for visualization. It is a nonlinear dimensionality reduction technique well-suited for embedding high-dimensional data for visualization in a low-dimensional space of two or three dimensions.

Similar objects are modeled by nearby points and dissimilar objects are modeled by distant points with high probability.

The t-SNE plot is a scatter plot that visualizes the clusters or groups of similar data points based on their similarity in the high-dimensional space. Each point in the plot corresponds to a specific sample in the dataset.

It can be used to understand the underlying structure of the data.

In [ ]:
hog_feature_extractor.tsne_transform()
tsne_result = hog_feature_extractor.tsne_transformed_data
tsne_result_data = {'tsne_x': tsne_result[:,0], 'tsne_y': tsne_result[:,1], 'face': train_y}

In [ ]:
fig, ax = plt.subplots(1)
sns.scatterplot(x ='tsne_x', y='tsne_y', data=tsne_result_data, s=100, hue='face')
ax.set_aspect('equal')
ax.legend(['Michael and Sarah', 'Jesse', 'Mila'], loc=3, title='Faces')

### 1.1.2. Discussion

The cluster representing Mila is distinctly separated from the rest of the data. However, there is noticeable overlap between the clusters of Jesse and Michael & Sarah. This is expected, as Jesse and Michael share similar facial features.


**SIFT Feature extractor**

SIFT stands for Scale-Invariant Feature Transform and it is a technique in computer vision used to detect and describe local features in images. It’s robust to changes in rotation, scale, and illumination. The process involves identifying key locations, assigning an orientation, and creating a descriptor based on local image gradients.

these are the steps:
1. Normalizing the image
2. Applying the SIFT detector
3. Returning the descriptors with corresponding keypoints

The descriptors can be used to train a model. However not all images have the same amount of descriptors.

In [ ]:
class SIFTFeatureExtractor(IdentityFeatureExtractor):
    
    def __init__(self, **params):
        self.params = params
        self.visual_representation = []
        self.tsne_transformed_data = None
        self.keypoints = []
        self.descriptors = []
        
    def transform(self, X):
        for instance in X:          
            instance = np.array(instance, dtype='uint8')
            instance_rsz = cv2.resize(instance, (64, 64)) 
             # Converting image to grayscale
            instance_rsz_gray = cv2.cvtColor(instance_rsz,cv2.COLOR_BGR2GRAY)
 
            # Applying SIFT detector
            sift = cv2.SIFT_create()
            # kp = sift.detect(instance_rsz_gray, None)
            kp, des = sift.detectAndCompute(instance_rsz_gray, None)
            
            if des is None:
                des = np.empty((0, 128))
 
            # Marking the keypoint on the image using circles
            sift_image_kp = cv2.drawKeypoints(instance_rsz_gray ,
                                  kp ,
                                  instance_rsz ,
                                  flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        
            self.visual_representation.append(sift_image_kp)
            self.keypoints.append(kp)
            self.descriptors.append(des)

In [ ]:
sift_feature_extractor = SIFTFeatureExtractor()
sift_feature_extractor.transform(train_X)

This vizualisation shows the detected keypoints by the SIFT detector on the image.

In [ ]:
plot_image_sequence(sift_feature_extractor.visual_representation, n=50, imgs_per_row=10)

### 1.1.3. Discussion

While SIFT is effective in feature detection, it proved challenging to apply in a classification context. The main issue encountered was the lack of consistency in the number of features returned by SIFT, making it extremely difficult to train a machine learning model. As a result, it was decided not to use SIFT further in the notebook.


## 1.2. Baseline 2: PCA feature extractor

**Reduce Dimension**

To begin with PCA, the image dataset is first converted into a 2D matrix. The images are initially transformed into grayscale, which reduces some of the dimensionality. This reduction in dimensionality helps make computations more efficient. Afterward, the images are flattened, creating the `trainingFaces` variable, which holds a list of flattened face vectors. Finally, mean-subtraction is performed to center the data around the origin, helping to reduce bias and improve the effectiveness of subsequent computations.

**SVD**

Once the data is preprocessed, the computation of eigenfaces begins. Singular Value Decomposition (SVD) is applied to the flattened faces. While SVD yields several results, the primary focus is on the U matrix (the left singular values), which contains the eigenfaces. While Eigenvalue Decomposition (EVD) could be an alternative, SVD is preferred due to its greater robustness, particularly in scenarios with noisy data or missing values.

The number of non-zero eigenvalues or singular values retained depends on the specific dataset and the desired level of information retention. A general rule of thumb is to keep enough components to explain a significant portion of the data's variance.

**Plot**

The plotted data displays the average of all training faces on the left and the first eigenface on the right. The first eigenface is derived from the U matrix obtained through SVD.


In [ ]:
nfaces = len(train_X)
m = FACE_SIZE[0]
n = FACE_SIZE[1]

gray_faces = np.array([cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2GRAY) for image in train_X])
trainingFaces = np.swapaxes(np.array([(np.swapaxes(x,0,1)).flatten() for x in gray_faces]),0,1)
avgFace = np.mean(trainingFaces,axis=1) # size n*m by 1

# Compute eigenfaces on mean-subtracted training data
X = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T
U, S, VT = np.linalg.svd(X,full_matrices=0)

fig1 = plt.figure()
ax1 = fig1.add_subplot(121)
img_avg = ax1.imshow(np.reshape(avgFace,(m,n)).T)
img_avg.set_cmap('gray')
plt.axis('off')

ax2 = fig1.add_subplot(122)
img_u1 = ax2.imshow(np.reshape(U[:,0],(m,n)).T)
img_u1.set_cmap('gray')
plt.axis('off')

plt.show()

The following class incorporates code to transform faces into their principal components and vice versa. Upon instantiation of this class, the number of components can be specified. Subsequently, the eigenfaces are computed based on the training data previously generated.

The `transform` method converts a list of faces into a list of their corresponding principal components.

Conversely, the `inverse_transform` method reverses this process, converting a list of principal components back into a reconstruction of the original faces.

In [ ]:
class PCAFeatureExtractor(IdentityFeatureExtractor):

    # converts face images to gray values and reshapes the data so that each column represents data of a face
    def __prepare_data__(self, X):
      grayX = np.array([cv2.cvtColor(x.astype(np.uint8), cv2.COLOR_RGB2GRAY) for x in X])
      flattenX = np.swapaxes(np.array([x.flatten() for x in grayX]),0,1)

      return flattenX


    def __init__(self, n_components):
        self.n_components = n_components

        X = self.__prepare_data__(train_X)
        self.avgX = np.mean(X,axis=1)

        X = X - np.tile(self.avgX, (X.shape[1],1)).T
        U, S, VT = np.linalg.svd(X,full_matrices=0)

        self.U = U.copy()

    def transform(self, Y):
      X = self.__prepare_data__(Y)
      X = X - np.tile(self.avgX, (X.shape[1],1)).T

      return np.swapaxes(self.U[:,:self.n_components].T @ X, 0, 1)

    def inverse_transform(self, X):
      return self.avgX + np.swapaxes(self.U[:,:self.n_components]  @ np.swapaxes(X, 0, 1),0,1)

### 1.2.1. Eigenface Plots

The following code demonstrates varying results when reconstructing a face using different numbers of components that PCA can utilize for reconstruction.

Through analysis of these results, it becomes evident that increasing the number of components leads to better reconstructions, which aligns with logical expectations. Beyond 50 components, PCA achieves a reconstruction closely resembling the original face. Consequently, it can be concluded that 50 components should suffice for classification purposes, as selecting a higher number may risk overfitting the model.

In [ ]:
# face reconstruction
testFace = train_X[60]
gray_face = cv2.cvtColor(testFace.astype(np.uint8), cv2.COLOR_RGB2GRAY)
plt.imshow(cv2.cvtColor(testFace.astype(np.uint8), cv2.COLOR_RGB2GRAY))
plt.set_cmap('gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

r_list = [5, 10, 15, 20, 30, 40, 50, 60, 70, 79]
mse_values = []
for r in r_list:
  feature_extractor = PCAFeatureExtractor(r)
  pca_features = feature_extractor.transform([testFace])
  reconFace = feature_extractor.inverse_transform(pca_features)
  mse = mean_squared_error(gray_face, np.reshape(reconFace,(m,n)))
  mse_values.append(mse)

  img = plt.imshow(np.reshape(reconFace,(m,n)))
  img.set_cmap('gray')
  plt.title('r = ' + str(r))
  plt.axis('off')
  plt.show()

This plot depicts the decrease in mean squared error (MSE) between the reconstructed face and the original face with an increasing number of components. Notably, the MSE consistently decreases as more components are used. By employing 50 components, a significant drop in MSE is observed. This trend underscores the effectiveness of using PCA for reconstruction tasks, where a higher number of components results in increasingly accurate reconstructions.

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(r_list, mse_values, marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Mean Squared Error')
plt.title('MSE vs. Number of Components')
plt.grid(True)
plt.show()

The following code constructs all eigenfaces for PCA with a total of 50 components, resulting in 50 eigenfaces. These eigenfaces are then plotted to provide a clearer understanding of their appearance.

In [ ]:
n_components = 50

nfaces = len(train_X)
m = FACE_SIZE[0]
n = FACE_SIZE[1]

gray_faces = np.array([cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2GRAY) for image in train_X])
trainingFaces = np.swapaxes(np.array([x.flatten() for x in gray_faces]),0,1)
avgFace = np.mean(trainingFaces,axis=1) # size n*m by 1

# Compute eigenfaces on mean-subtracted training data
X = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T
U, S, VT = np.linalg.svd(X,full_matrices=0)

In [ ]:
plot_image_sequence([np.reshape(x, FACE_SIZE) for x in np.swapaxes(U[:,:n_components],0,1)], n=n_components, imgs_per_row=10)

In the face-features plot visualization, the faces are positioned based on the eigenface values they represent. This plot allows for a visual assessment of which faces have comparable representations for the first two eigenfaces when transformed by PCA. By observing the spacing between PCA features, interpretation becomes more intuitive and straightforward.

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

gray_faces = np.array([cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2GRAY) for image in train_X])
trainingFaces = np.swapaxes(np.array([x.flatten() for x in gray_faces]),0,1)
avgY = np.mean(trainingFaces,axis=1) # size n*m by 1
Y = trainingFaces - np.tile(avgY,(trainingFaces.shape[1],1)).T

features = np.swapaxes(U[:,:2].T @ Y, 0, 1)

fig, ax = plt.subplots()
ax.scatter(features[:,0], features[:,1])

for i, y in enumerate(gray_faces):
    ab = AnnotationBbox(OffsetImage(np.reshape(y.astype(np.uint8), (100, 100)), zoom=0.1, cmap='gray'),
                        (features[i, 0], features[i, 1]), frameon=False)
    ax.add_artist(ab)

plt.xlabel("Eigenface 1")
plt.ylabel("Eigenface 2")
plt.show()

### 1.2.2. Feature Space Plots
The next plot illustrates how the first two features are distributed when transformed with PCA. In this visualization, the features of person 1 are depicted in red, while those of person 2 are represented in black. By examining this visualization, it becomes apparent that the data points are almost linearly separable, suggesting that a simple linear boundary between the data points of each label could effectively classify each face. This observation holds promise for subsequent face classification tasks.

In [ ]:
PCA_features = PCAFeatureExtractor(10).transform(train_X)

f1 = 0
f2 = 1

p1 = 1
p2 = 2

plt.plot(PCA_features[train_y == p1][:,f1], PCA_features[train_y == p1][:,f2],'d',color='k',label='Person 0')
plt.plot(PCA_features[train_y == p2][:,f1], PCA_features[train_y == p2][:,f2],'^',color='r',label='Person 1')

plt.legend()
plt.show()

In the following PCA plot, the data is visualized in three dimensions, with each axis representing a different feature from PCA. The plot clearly shows distinct clusters of data points with the same labels, which suggests a strong foundation for future face classification tasks.

In [ ]:
PCA_features = PCAFeatureExtractor(10).transform(train_X)

f1 = 0
f2 = 1
f3 = 2

p1 = 1
p2 = 2

ax = plt.axes(projection='3d')
for p in range(3):
  ax.scatter(PCA_features[train_y == p][:,f1], PCA_features[train_y == p][:,f2], PCA_features[train_y == p][:,f3], color=['r','g','b'][p], label=f"Person {p}");

plt.legend()
plt.show()

### 1.2.3. Discussion

Principal Component Analysis (PCA) applied to facial image datasets provides crucial insights and prepares the data for subsequent tasks like classification. Through visualizations and analyses, the process of dimensionality reduction, eigenface computation, and the potential benefits of PCA in classification were examined.

PCA proves to be a valuable tool for reducing dimensionality and extracting features from facial image datasets. By transforming the data while retaining key information, PCA establishes a solid foundation for future classification tasks, enabling more efficient and accurate facial image analysis.


# 2. Evaluation Metrics
## 2.0. Example: Accuracy
As example metric I take the accuracy. Informally, accuracy is the proportion of correct predictions over the total amount of predictions. It is used a lot in classification but it certainly has its disadvantages...

In [ ]:
from sklearn.metrics import accuracy_score

The classification_report function is also imported from sklearn because accuracy does not tells us a lot about how robust the model is. With this function we can look at the precision, recall  and f1-score of each class.

In [ ]:
from sklearn.metrics import classification_report

# 3. Classifiers
## 3.0. Example: The *'not so smart'* classifier
This random classifier is not very complicated. It makes predictions at random, based on the distribution obseved in the training set. **It thus assumes** that the class labels of the test set will be distributed similarly to the training set.

In [ ]:
class RandomClassificationModel:
    """Random classifier, draws a random sample based on class distribution observed 
    during training."""
    
    def fit(self, X, y):
        """Adjusts the class ratio instance variable to the one observed in y. 

        Parameters
        ----------
        X : tensor
            Training set
        y : array
            Training set labels

        Returns
        -------
        self : RandomClassificationModel
        """
        
        self.classes, self.class_ratio = np.unique(y, return_counts=True)
        self.class_ratio = self.class_ratio / self.class_ratio.sum()
        return self
        
    def predict(self, X):
        """Samples labels for the input data. 

        Parameters
        ----------
        X : tensor
            dataset
            
        Returns
        -------
        y_star : array
            'Predicted' labels
        """

        np.random.seed(0)
        return np.random.choice(self.classes, size = X.shape[0], p=self.class_ratio)
    
    def __call__(self, X):
        return self.predict(X)
    

## 3.1 Nearest Neighbours

One of the first classifiers explored was Nearest Neighbours. This is a very classic, well-known classifier. However, the performance of both the HOG and PCA feature extractors wasn't that good. A combination of the HOG and the PCA features was additionally also explored.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

feature_extractor_PCA = PCAFeatureExtractor(100)
pca_result = feature_extractor_PCA.transform(train_X)
feature_extractor_HOG = HOGFeatureExtractor()
HOG_result = feature_extractor_HOG.transform(train_X)

# HOG has big dimension so I don't know how much this helps
features = np.hstack((pca_result, HOG_result))

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(features, train_y)

print(f'Cross validation:\n {cross_val_score(neigh, features, train_y, cv=10)}')

'''
feature_extractor_PCA = PCAFeatureExtractor(100)
pca_result = feature_extractor_PCA.transform(test_X)
feature_extractor_HOG = HOGFeatureExtractor()
HOG_result = feature_extractor_HOG.transform(test_X)

features_test = np.hstack((pca_result, HOG_result))

test_y_star = neigh.predict(features_test)

# evaluate performance of the model on the training set
train_y_star = neigh.predict(features)

"The performance on the Test set is {:.2f}. This however, does not tell us much about the actual performance (generalisability).".format(
    accuracy_score(train_y, train_y_star))
'''

## 3.2 Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression

class LRClassificationModel:
    def __init__(self):
        self.lr_model = LogisticRegression(C=1.0, 
                                           verbose=0, 
                                           class_weight='balanced', 
                                           dual=False,
                                           fit_intercept=False, 
                                           intercept_scaling=3, 
                                           max_iter=200, 
                                           multi_class='auto', 
                                           n_jobs=None, penalty='l2', 
                                           random_state=None, 
                                           solver='lbfgs', 
                                           tol=0.01)

    def fit(self, X, y):
        #X_train = self.get_feature_extractor.transform(X)
        self.lr_model.fit(X, y)

    def predict(self, X):
        #X_test = self.get_feature_extractor.transform(X)
        return self.lr_model.predict(X)
    
    def predict_proba(self, X):
        #X_test = self.feature_extractor_instance.transform(X)
        return self.lr_model.predict_proba(X)
    
    def get_model(self):
        return self.lr_model
        

**Hyper parameters**

In the next block of code grid search is done to find the optimal hyper parameters for the model. This only had to be done once and that is why it is been commented out.

In [ ]:
# random grid search to find the best hyper parameters for Logistic Regression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
'''
#feature_extractor = PCAFeatureExtractor(100)
#pca_features = feature_extractor.transform(train_X)
hog_feature_extractor = HOGFeatureExtractor()
hog_features = hog_feature_extractor.transform(train_X)

# Use a random grid search to find the best hyper parameters

param_dist = {
    'C': [0.1, 1.0, 2.0, 5.0, 10.0],  # Regularization parameter
    'tol': [0.1, 0.01, 0.001, 0.0001],
    'solver': ['lbfgs', 'liblinear', 'saga', 'newton-cg', 'sag'],  # Solver options
    'max_iter': [50, 100, 200, 500, 1000, 2000],  # Maximum number of iterations
    'penalty' : ['l2'],
    'verbose' : [0, 1, 2]
    }
# Initialize logistic regression model
lr_model = LogisticRegression(class_weight='balanced', multi_class='auto', fit_intercept=False)
# Perform grid search
# Perform randomized grid search
random_search = RandomizedSearchCV(lr_model, param_distributions=param_dist, n_iter=500,
                                   scoring='accuracy', cv=5, verbose=0, random_state=42)
random_search.fit(pca_features, train_y)
# Get best hyperparameters
best_params = random_search.best_params_
print(f"Best hyperparameters: {best_params}")'''

## 3.3 Support Vector Machine
Both SVC and lineairSVC have been explored.

In [ ]:
from sklearn.svm import SVC

class SVCClassificationModel:
    def __init__(self):
        self.svm_model = SVC(C=2.086, 
                             class_weight=None, 
                             coef0=3.929, 
                             decision_function_shape='ovo', 
                             degree=3, gamma='scale', 
                             kernel='rbf', 
                             max_iter=10000, 
                             shrinking=False, 
                             tol=0.0970105895663254)


    def fit(self, X, y):
        self.svm_model.fit(X, y)

    def predict(self, X):
        return self.svm_model.predict(X)
    
    def predict_proba(self, X):
        return self.svm_model.predict_proba(X)
    
    def get_model(self):
        return self.svm_model

In [ ]:
from sklearn.svm import LinearSVC

class LineairSVCClassificationModel:
    def __init__(self):
        self.svm_model =  LinearSVC(C=3.65,
                                    class_weight='balanced', 
                                    fit_intercept=True, 
                                    intercept_scaling=9.65, 
                                    max_iter=150, tol=0.096)

    def fit(self, X, y):
        self.svm_model.fit(X, y)

    def predict(self, X):
        return self.svm_model.predict(X)
    
    def predict_proba(self, X):
        return self.svm_model.predict_proba(X)
    
    def get_model(self):
        return self.svm_model

**Hyper parameters**

In the next block of code did grid search is done to find the optimal hyper parameters for the model. This only had to be done once and that is why it is been commented out.

In [ ]:
# random parameter search for SMV 

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
'''
#augmented_train_X
#augmented_train_y 
#feature_extractor = PCAFeatureExtractor(100)
#pca_features_train = feature_extractor.transform(reduced_train_X)
#hog_feature_extractor = HOGFeatureExtractor()
#hog_features_train = hog_feature_extractor.transform(augmented_train_X)
feature_extractor = PCAFeatureExtractor(100)
pca_features_train = feature_extractor.transform(augmented_train_X)



svm_model = LinearSVC()
param_distributions = {"C": uniform(1, 10), 
                       "tol": uniform(0.0001, 0.1),
                       "max_iter": [100, 500, 1000, 2000],
                       "fit_intercept": [True, False],
                       "intercept_scaling": uniform(0.1, 10),
                       "class_weight": [None, "balanced"]}

random_search = RandomizedSearchCV(svm_model, param_distributions, n_iter=200, cv=5, verbose=2)
random_search.fit(hog_features_train, augmented_train_y)

print(f"Best hyperparameters: {random_search.best_params_}")'''

## 3.4 Convolutional neural network
Tested with the convolutional neural network but did not see any good results from it so did not look further into it.

In [ ]:
'''
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout

augmented_train_X, augmented_train_y = generate_augmented_data(train_X, train_y)

# building the input vector from the augmented data
X_train = augmented_train_X.reshape(augmented_train_X.shape[0], 100, 100, 3)
X_train = X_train.astype('float32')

# normalizing the data to help with the training
X_train /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 3
print("Shape before one-hot encoding: ", augmented_train_y.shape)
Y_train = to_categorical(augmented_train_y, n_classes)
#print("Shape after one-hot encoding: ", Y_train.shape)

conv_model = Sequential()
conv_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(100,100,3)))
conv_model.add(MaxPool2D(pool_size=(2,2)))
conv_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
conv_model.add(MaxPool2D(pool_size=(2,2)))
conv_model.add(Dropout(0.25))
conv_model.add(Flatten())
conv_model.add(Dense(128, activation='relu'))
conv_model.add(Dropout(0.5))
conv_model.add(Dense(n_classes, activation='softmax'))  # Change to match the number of classes

# compiling the sequential model
conv_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
conv_model.fit(X_train, Y_train, batch_size=32, epochs=30)'''

## 3.5 XGBoost
Lastly, XGBoost was also explored, but did not see any good results from it so did not look further into it. 

In [ ]:
'''
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Convert data into DMatrix format for XGBoost
dtrain = xgb.DMatrix(pca_features, label=train_y)
# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # multiclass classification
    'num_class': 3,                 # number of classes
    'eval_metric': 'merror'         # evaluation metric
}
# Train the XGBoost model
num_rounds = 100
#xgb_model = xgb.train(params, dtrain, num_rounds)
'''

# 4. Experiments


## 4.0. Example: basic pipeline
The basic pipeline takes any input and samples a label based on the class label distribution of the training set. As expected the performance is very poor, predicting approximately 1/4 correctly on the training set. There is a lot of room for improvement. 

**Data augmentation** 

To maximize the amount of training data available to our classifiers, augmented data is added to the training set. This augmented data is added in order to help generalize the classifier better by including more varied training data. This is a well-known technique used when training data is sparse. When used, observed a slight improvement in performance.

In [ ]:
# adds an augmented version of each remaining image
import numpy as np

# updated 
def generate_augmented_data(train_X, train_y):
    augmented_train_X = []
    augmented_train_y = []

    for i in range(len(train_X)):
        augmented_train_X.append(train_X[i])
        augmented_train_y.append(train_y[i])
        
        for _ in range(20):
            aug = random_augmentations(train_X[i], p=1)
            augmented_train_X.append(aug)
            augmented_train_y.append(train_y[i])

    # Convert the lists to ndarrays before returning
    augmented_train_X = np.stack(augmented_train_X)
    augmented_train_y = np.array(augmented_train_y)

    return augmented_train_X, augmented_train_y

In [ ]:
augmented_train_X = np.zeros((0, train_X.shape[1], train_X.shape[2], train_X.shape[3])).astype(np.uint8)
augmented_train_y = np.zeros((0)).astype(np.uint8)
samples = 40
for index, X in enumerate(train_X):
    augmented_train_X = np.concatenate((augmented_train_X, [random_augmentations(X) for i in range(samples)]), axis=0)
    augmented_train_y = np.concatenate((augmented_train_y, [train_y[index] for i in range(samples)]), axis=0)

In [ ]:
train_X_augmented, train_y_augmented = generate_augmented_data(train_X, train_y)

## 4.1. Review of Classifiers

Various classifiers were evaluated in combination with different feature extraction methods, using a small validation set for demonstration purposes. Due to the limited size of the validation set, the results obtained may not fully reflect the model's performance in a real-world scenario. During testing, it was observed that Logistic Regression (LR) with Histogram of Oriented Gradients (HOG) performed the best as a standalone model for classifying faces. However, combining Support Vector Machine (SVM) with Principal Component Analysis (PCA) and LR with HOG resulted in a higher score than LR with HOG alone. It is important to note that in a real-world context, a different combination of classifier and feature extractor may yield better results than those observed on the validation set.


#### Helper function to review classifiers

In [ ]:
import warnings
warnings.filterwarnings("ignore")
def best_features_for_model(model, train_X, train_y, test_X, test_y):
    # different trainingsdata representations
        #PCA 80
    feature_extractor_PCA = PCAFeatureExtractor(80)
    pca_result_80 = feature_extractor_PCA.transform(train_X)
    pca_result_80_test = feature_extractor_PCA.transform(test_X)

    
        #PCA 50
    feature_extractor_PCA = PCAFeatureExtractor(50)
    pca_result_50 = feature_extractor_PCA.transform(train_X)
    pca_result_50_test = feature_extractor_PCA.transform(test_X)

    
        # HOG transfrom
    feature_extractor_HOG = HOGFeatureExtractor()
    HOG_result = feature_extractor_HOG.transform(train_X)
    feature_extractor_HOG = HOGFeatureExtractor()
    HOG_result_test = feature_extractor_HOG.transform(test_X)

        # Combinations
    combined = np.hstack((pca_result_80, HOG_result))
    combined_test = np.hstack((pca_result_80_test, HOG_result_test))
    
    
        # Augmented data PCA
    train_X_augmented, train_y_augmented = generate_augmented_data(train_X, train_y)
    feature_extractor_PCA = PCAFeatureExtractor(80)
    pca_result_80_aug = feature_extractor_PCA.transform(train_X_augmented)
  
    
        # Augmented data HOG
    feature_extractor_HOG = HOGFeatureExtractor()
    HOG_result_aug = feature_extractor_HOG.transform(train_X_augmented)
    
        # Combinations augmented
    combined_aug = np.hstack((pca_result_80_aug, HOG_result_aug))    
    
    train_sets = [{'Name trainset' : 'PCA 80', 'train_X' : pca_result_80, 'train_y' : train_y,'test_X' : pca_result_80_test, 'test_y' : test_y},
                 {'Name trainset' : 'PCA 50', 'train_X' : pca_result_50, 'train_y' : train_y,'test_X' : pca_result_50_test, 'test_y' : test_y},
                 {'Name trainset' : 'Hog', 'train_X' : HOG_result, 'train_y' : train_y,'test_X' : HOG_result_test, 'test_y' : test_y},
                 {'Name trainset' : 'Combined', 'train_X' : combined, 'train_y' : train_y, 'test_X' : combined_test, 'test_y' : test_y},
                 {'Name trainset' : 'Augmented PCA', 'train_X' : pca_result_80_aug, 'train_y' : train_y_augmented, 'test_X' : pca_result_80_test, 'test_y' : test_y},
                 {'Name trainset' : 'Augmented HOG', 'train_X' : HOG_result_aug, 'train_y' : train_y_augmented, 'test_X' : HOG_result_test, 'test_y' : test_y},
                 {'Name trainset' : 'Augmented Combined', 'train_X' : combined_aug, 'train_y' : train_y_augmented, 'test_X' : combined_test, 'test_y' : test_y}

                 ]
    scores =[]
    
    for train_set in train_sets:
        print(train_set['Name trainset'])
        model.fit(train_set['train_X'], train_set['train_y'])
        train_acc = accuracy_score(model.predict(train_set['test_X']), train_set['test_y'])
        scores.append({ 'train_set' : train_set['Name trainset'], 'test_acc' : train_acc})
        print(classification_report(model.predict(train_set['test_X']), train_set['test_y'], labels=[0, 1, 2]))
    print(scores)

**Logistic Regression**

The results indicate that the HOG feature extraction method provides the best performance when used with Logistic Regression. Both the original and augmented HOG methods achieved high accuracy (0.95 and 0.90 respectively), precision, recall, and F1-score across all classes.

In contrast, the PCA and Combined methods, both original and augmented, showed lower performance metrics.

Therefore, for this particular task and dataset, Logistic Regression paired with HOG appears to be the most effective approach.

Keep in mind that the validation set is very small. So the results on the test set may vary.

In [ ]:
validation_size = 20

combined = list(zip(train_X, train_y))
random.shuffle(combined)
X, y = zip(*combined)

validation_X = X[-validation_size:]
validation_y = y[-validation_size:]

X = X[:-validation_size]
y = y[:-validation_size]

lr_model = LRClassificationModel()
best_features_for_model(lr_model, X, y, validation_X, validation_y)

**Support Vector Classifier**

The SVC performs best with the HOG feature extraction method, both in its original and augmented forms, achieving an accuracy of 0.90 and 0.95 respectively.

The Combined methods, original and augmented, showed lower performance metrics. But SVC also works well with PCA.

Thus, for this particular task and dataset, SVC paired with HOG appears to be the most effective approach. 

Keep in mind that the validation set is very small. So the results on the test set may vary.

In [ ]:
validation_size = 20

combined = list(zip(train_X, train_y))
random.shuffle(combined)
X, y = zip(*combined)

validation_X = X[-validation_size:]
validation_y = y[-validation_size:]

X = X[:-validation_size]
y = y[:-validation_size]

lr_model = SVCClassificationModel()
best_features_for_model(lr_model, X, y, validation_X, validation_y)

**K-nearest neighbor**

The KNN classifier shows the best performance with the HOG feature extraction method, achieving an accuracy of 0.80.

The PCA and Combined methods, both original and augmented, showed similar performance metrics with an accuracy of 0.80 and 0.70 respectively.

However, the performance of KNN with the Augmented HOG method dropped slightly to an accuracy of 0.70.

In conclusion, for this particular task and dataset, KNN paired with HOG appears to be the most effective approach.

Keep in mind that the validation set is very small. So the results on the test set may vary.

In [ ]:
validation_size = 20

combined = list(zip(train_X, train_y))
random.shuffle(combined)
X, y = zip(*combined)

validation_X = X[-validation_size:]
validation_y = y[-validation_size:]

X = X[:-validation_size]
y = y[:-validation_size]

lr_model = KNeighborsClassifier(n_neighbors=5)
best_features_for_model(lr_model, X, y, validation_X, validation_y)

In [ ]:
test_X_indices = []
for i, faces in enumerate(test_X_faces):
    for _ in faces:
        test_X_indices.append(i)
test_X_indices = np.array(test_X_indices)

In [ ]:
test_X = []
for faces in test_X_faces:
    for face in faces:
        test_X.append(face)
test_X = np.array(test_X).astype(np.int64)

In the following code block a Support Vector Classifier (SVC) is trained using features extracted from images through Principal Component Analysis (PCA). The augmented training data is utilized to fit this model. Once trained, the model is employed to make predictions on the test set. These predictions include confidence scores for each label, indicating the model's level of certainty.

In [ ]:
# PCA (SVC)
pca_feature_extractor = PCAFeatureExtractor(50)
svc = SVC(probability=True)

pca_features_train = pca_feature_extractor.transform(train_X_augmented)
svc.fit(pca_features_train, train_y_augmented)

# do cross validation, commented out to reduce output 
#print(f'Cross validation:\n {cross_val_score(svc, pca_features_train, train_y_augmented, cv=10)}')

pca_features_test = pca_feature_extractor.transform(test_X)
test_y_prob_pca = svc.predict_proba(pca_features_test)

In the following code block a Logistic Regression Classifier (LRC) is trained using features extracted from images through Principal Component Analysis (PCA). The augmented training data is alse utilized to fit this model. Once trained, the model is employed to make predictions on the test set. These predictions include confidence scores for each label, such as the output obtained by the previous model.

In [ ]:
# HOG (LR)
lrc = LRClassificationModel()

hog_feature_extractor = HOGFeatureExtractor()
hog_features_train = hog_feature_extractor.transform(train_X_augmented)
lrc.fit(hog_features_train, train_y_augmented)
# do cross validation, commented out to reduce output 
#print(f'Cross validation:\n {cross_val_score(lrc.get_model(), hog_features_train, train_y_augmented, cv=10)}')

hog_feature_extractor = HOGFeatureExtractor() # Have to make new isntance 
hog_features_test = hog_feature_extractor.transform(test_X)
test_y_prob_hog = lrc.predict_proba(hog_features_test)

The method `classify_rules` accepts a list of predictions generated by multiple models. Each model makes predictions for all faces extracted from the image and assigns a confidence/probability to these predictions. By aggregating this data, the `classify_rules` method selects the prediction with the highest confidence from all models. If other models also exhibit high confidence in the chosen prediction, it is returned. Otherwise, the method defaults to predicting class 0.

This `classify_rules` method is then applied to each face in the test set.

In [ ]:
# Select prediction from model using confidence
def classify_rules(models_preds_probs):
    # Find model that has highest confidence about prediction
    best_model_index = np.argmax([model_preds_probs.max() for model_preds_probs in models_preds_probs])
    best_model_preds_probs = models_preds_probs[best_model_index]
    
    # Find face for which the model has the highest confidence about prediction
    face_index = np.argmax([preds_probs.max() for preds_probs in best_model_preds_probs])
    best_pred_prob = best_model_preds_probs[face_index]
    
    # Make prediction by choosing the one with highest confidence from all models
    prediction = np.argmax(best_pred_prob)
    
    # Predict the other class (0) if other models don't agree with model with highest confidence about prediction
    min_conf_prediction_face = min([model_preds_probs[face_index][prediction] for model_preds_probs in models_preds_probs])
    if min_conf_prediction_face < 0.6:
        return 0
    
    return prediction

# combine the results from both classifers based on confidence
test_y_star = [classify_rules([test_y_prob_hog[test_X_indices == i], test_y_prob_pca[test_X_indices == i]]) for i in range(len(test_X_faces))]

During testing, it became evident that Logistic Regression (LR) with Histogram of Oriented Gradients (HOG) outperformed as a standalone model in classifying faces. However, when combining both Support Vector Machine (SVM) with Principal Component Analysis (PCA) and LR with HOG, the combined score surpassed that of LR with HOG alone.

# 5. Publishing best results

In [ ]:
submission = test.copy().drop('img', axis = 1)
submission['class'] = test_y_star

submission

In [ ]:
submission.to_csv('submission.csv')

# 6. Discussion

In summary, the following contributions were made:

**Preprocessing**

The project began with experiments involving various preprocessing techniques. Despite efforts to fine-tune the parameters of the provided HAAR detector to capture more faces, significant limitations were encountered, including an increase in false positives. An alternative approach using a pre-trained DNN was also tested, but it failed to outperform the HAAR detector and resulted in suboptimal cropping, particularly cutting out parts of facial features, such as chins, across multiple images. Ultimately, the HAAR detector was retained due to its comparatively better performance.

**Feature Extractors**

Three distinct feature extraction methods were explored: Histogram of Oriented Gradients (HOG), Scale-Invariant Feature Transform (SIFT), and Principal Component Analysis (PCA).

- HOG counts the occurrences of gradient orientation in localized sections of an image, focusing on the structure or shape of the object.
- SIFT, although explored, was not integrated into the classifiers due to challenges with varying feature counts per image, which complicated feature manipulation.
- PCA, which used Singular Value Decomposition (SVD) to derive eigenfaces from images, proved to be a promising approach. After fine-tuning the number of components, it was determined that using 50 components produced the best results.

**Metrics**

The classifiers were evaluated using a variety of metrics. Initially, accuracy was used, but to gain more detailed insights, additional metrics such as `classification_report` and `cross_validation` from sklearn were employed. These more refined metrics, particularly `classification_report`, offered valuable precision and recall information for each individual label.

**Classifier**

The integration of the previously discussed techniques led to the implementation of the classifiers. Different models were tested alongside various feature extractors to determine the most effective combination. The best-performing approach was identified as a dual-model strategy, combining an SVC classifier for PCA features and a logistic regression classifier for HOG features. Image classification was based on the model with the highest confidence in its assigned label. Additionally, training data augmentation was explored, which proved to be beneficial for improving model generalization and demonstrated effectivenes
